In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

pd.set_option('future.no_silent_downcasting', True)

In [120]:
df_train = pd.read_csv('./data/train.csv', index_col='Id')
df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [121]:
def load_descriptions(file_path):
    descriptions = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        current_key = None
        current_description = []

        for line in lines:
            line = line.strip()
            if line:  # Only process non-empty lines
                if not line.startswith(" "):  # New description starts
                    # Save the previous description if it exists
                    if current_key and current_description:
                        descriptions[current_key] = " ".join(current_description).strip()

                    # Split line and check for proper formatting
                    parts = line.split(":")
                    if len(parts) >= 2:
                        current_key = parts[0].strip()
                        current_description = [parts[1].strip()]
                    else:
                        current_key = None  # Reset current_key to avoid carrying over
                        current_description = []
                elif current_key:  # Continuing a description
                    current_description.append(line.strip())

        # Save the last description if exists
        if current_key and current_description:
            descriptions[current_key] = " ".join(current_description).strip()

    return descriptions

descriptions = load_descriptions('./data/data_description.txt')

In [122]:
def summarize_dataframe(df, descriptions):
    summary = {
        'Column': [],
        'Description': [],
        'Data Type': [],
        'Null Count': [],
        'Unique Values': [],
        'Range': [],
        'Mean': [],
        'Standard Deviation': [],
    }

    for col in df.columns:
        summary['Column'].append(col)
        summary['Data Type'].append(df[col].dtype)
        summary['Null Count'].append(df[col].isnull().sum())

         # Show list of unique values if categorical
        if df[col].dtype.kind not in 'iufc':
            if df[col].dropna().nunique() < 20:
                # Show unique values by count (descending order)
                summary['Unique Values'].append(df[col].value_counts(dropna=True).index.tolist())
            else:
                summary['Unique Values'].append(None)
        else:
            summary['Unique Values'].append(None)

        if pd.api.types.is_numeric_dtype(df[col]):
            summary['Range'].append((df[col].min(), df[col].max()))
            summary['Mean'].append(round(df[col].mean(), 2))
            summary['Standard Deviation'].append(round(df[col].std(), 2))
        else:
            summary['Range'].append(None)
            summary['Mean'].append(None)
            summary['Standard Deviation'].append(None)

        # Add description if available
        summary['Description'].append(descriptions.get(col, "No description available"))

    summary_df = pd.DataFrame(summary)
    return summary_df

summarize_dataframe(df_train, descriptions)

,Column,Description,Data Type,Null Count,Unique Values,Range,Mean,Standard Deviation
0,MSSubClass,Identifies the type of dwelling involved in th...,int64,0,None,"(20, 190)",56.90,42.30
1,MSZoning,Identifies the general zoning classification o...,object,0,"[RL, RM, FV, RH, C (all)]",None,NaN,NaN
2,LotFrontage,Linear feet of street connected to property,float64,259,None,"(21.0, 313.0)",70.05,24.28
3,LotArea,Lot size in square feet,int64,0,None,"(1300, 215245)",10516.83,9981.26
4,Street,Type of road access to property,object,0,"[Pave, Grvl]",None,NaN,NaN
...,...,...,...,...,...,...,...,...
75,MoSold,Month Sold (MM),int64,0,None,"(1, 12)",6.32,2.70
76,YrSold,Year Sold (YYYY),int64,0,None,"(2006, 2010)",2007.82,1.33
77,SaleType,Type of sale,object,0,"[WD, New, COD, ConLD, ConLI, ConLw, CWD, Oth, ...",None,NaN,NaN
78,SaleCondition,Condition of sale,object,0,"[Normal, Partial, Abnorml, Family, Alloca, Adj...",None,NaN,NaN


### Insights

- `MSSubClass` represents a categorical variable, despite being represented as integers.
- `CentralAir` can be better represented as a boolean variable.

In [123]:
# Handle missing values in each column

# 1. Handle LotFrontage
df_train['LotFrontage'] = df_train.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(x.median())
)

# 2. Handle Alley
df_train['Alley'] = df_train['Alley'].fillna('None')

# 3. Handle MasVnrType and MasVnrArea
df_train['MasVnrType'] = df_train['MasVnrType'].fillna('None')
df_train.loc[df_train['MasVnrType'] == 'None', 'MasVnrArea'] = 0
df_train['MasVnrArea'] = df_train['MasVnrArea'].fillna(df_train['MasVnrArea'].median())

# 4. Handle Basement Features
basement_cols = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
for col in basement_cols:
    df_train[col] = df_train[col].fillna('NA')

# 5. Handle Electrical
df_train['Electrical'] = df_train['Electrical'].fillna(df_train['Electrical'].mode()[0])

# 6. Handle FireplaceQu
df_train['FireplaceQu'] = df_train['FireplaceQu'].fillna('NA')

# 7. Handle Garage Features
garage_categorical = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
for col in garage_categorical:
    df_train[col] = df_train[col].fillna('NA')
df_train.loc[df_train['GarageType'] == 'NA', 'GarageYrBlt'] = 0
df_train['GarageYrBlt'] = df_train['GarageYrBlt'].fillna(df_train['GarageYrBlt'].median())

# 8. Handle PoolQC
df_train['PoolQC'] = df_train['PoolQC'].fillna('NA')

# 9. Handle Fence
df_train['Fence'] = df_train['Fence'].fillna('NA')

# 10. Handle MiscFeature
df_train['MiscFeature'] = df_train['MiscFeature'].fillna('NA')

In [124]:
# Verify no missing values remain
print(df_train.isnull().sum()[df_train.isnull().sum() > 0])

Series([], dtype: int64)


In [125]:
# Convert CentralAir to boolean
def map_central_air(X):
    X['CentralAir'] = X['CentralAir'].map({
        'Y': True,
        'N': False
    }).fillna(False)
    return X

In [127]:
# Define ordinal and nominal columns
ordinal_columns = [
    "LotShape", "LandContour", "Utilities", "LandSlope", "HouseStyle",
    "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "BsmtExposure",
    "BsmtFinType1", "BsmtFinType2", "HeatingQC", "Electrical", "KitchenQual",
    "Functional", "FireplaceQu", "GarageFinish", "GarageQual", "GarageCond",
    "PavedDrive", "PoolQC", "Fence"
]

nominal_columns = [
    "MSSubClass", "MSZoning", "Alley", "LotConfig", "Neighborhood",
    "Condition1", "Condition2", "RoofStyle", "RoofMatl", "Exterior1st",
    "Exterior2nd", "MasVnrType", "Heating", "MiscFeature", "SaleType",
    "SaleCondition", "Street", "BldgType", "Foundation", "GarageType"
]

# Identify numeric columns
numeric_cols = df_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols.remove('MSSubClass')
numeric_cols.remove('SalePrice')  # Exclude target variable

In [128]:
nominal_columns_categories = {col: map_central_air(df_train)[col].unique().tolist() for col in nominal_columns}
ordinal_columns_categories = {
    'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
    'LandContour': ['Lvl', 'Bnk', 'Low', 'HLS'],
    'Utilities': ['ELO', 'NoSeWa', 'NoSeWr', 'AllPub'],
    'LandSlope': ['Gtl', 'Mod', 'Sev'],
    'HouseStyle': ['1Story', '1.5Unf', '1.5Fin', '2Story', '2.5Unf', '2.5Fin', 'SFoyer', 'SLvl'],
    'ExterQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'ExterCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtQual': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtCond': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtExposure': ['NA', 'No', 'Mn', 'Av', 'Gd'],
    'BsmtFinType1': ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'BsmtFinType2': ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'HeatingQC': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'Electrical': ['FuseP', 'FuseF', 'FuseA', 'SBrkr', 'Mix'],
    'KitchenQual': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'Functional': ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'],
    'FireplaceQu': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'GarageFinish': ['NA', 'Unf', 'RFn', 'Fin'],
    'GarageQual': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'GarageCond': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'PavedDrive': ['N', 'P', 'Y'],
    'PoolQC': ['NA', 'Fa', 'TA', 'Gd', 'Ex'],
    'Fence': ['NA', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']
}

In [129]:
# Create the preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ('custom_mapping', FunctionTransformer(map_central_air, validate=False), ['CentralAir']),

    # Ordinal Features: Impute missing values and encode using OrdinalEncoder
    ('ordinal', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
        ('ordinal_encoder', OrdinalEncoder(categories=[ordinal_columns_categories[col] for col in ordinal_columns]))  # Apply Ordinal Encoding
    ]), ordinal_columns),

    # Nominal Features: Impute and OneHotEncode
    ('nominal', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
        ('onehot', OneHotEncoder(categories=[nominal_columns_categories[col] for col in nominal_columns], handle_unknown='ignore'))
    ]), nominal_columns),

    # Numeric Features: Impute missing values and scale
    ('numeric', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
        ('scaler', StandardScaler())  # Standard scaling
    ]), numeric_cols)
])

In [130]:
# Full pipeline: Preprocessing + Feature Selection + Linear Regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocessing step
    ('feature_selection', SelectKBest(score_func=f_regression, k=20)),  # Select top 20 features
    ('regressor', LinearRegression())  # Linear regression model
])

In [131]:
# Split the data into train and test sets
X = df_train.drop(columns=['SalePrice'])
y = df_train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

In [132]:
# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('custom_mapping',
                                                  FunctionTransformer(func=<function map_central_air at 0x7be26de92a70>),
                                                  ['CentralAir']),
                                                 ('ordinal',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinal_encoder',
                                                                   OrdinalEncoder(categories=[['Reg',
                                                                                               'IR1',
                                                                                               'IR2',
                                                                                               'IR3'],
                                                                                              ['Lvl',
                                                                                               'Bnk',
                                                                                               'Low',
                                                                                               'HLS'],
                                                                                              ['ELO',...
                                                   'GrLivArea', 'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBath', 'BedroomAbvGr',
                                                   'KitchenAbvGr',
                                                   'TotRmsAbvGrd', 'Fireplaces',
                                                   'GarageYrBlt', 'GarageCars',
                                                   'GarageArea', 'WoodDeckSF',
                                                   'OpenPorchSF',
                                                   'EnclosedPorch', '3SsnPorch', ...])])),
                ('feature_selection',
                 SelectKBest(k=20,
                             score_func=<function f_regression at 0x7be2739b9510>)),
                ('regressor', LinearRegression())])

In [133]:
# Make predictions
y_pred = pipeline.predict(X_test)

In [134]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

Mean Squared Error: 1038556538.1219907
R² Score: 0.8394882109863718


In [135]:
df_test = pd.read_csv('./data/test.csv', index_col='Id')
df_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [136]:
df_test['SalePrice'] = pipeline.predict(df_test)
df_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,104501.788853
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,154329.084868
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,167570.214780
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,182207.404044
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,209428.443159


In [137]:
df_test['SalePrice']

,SalePrice
Id,
1461,104501.788853
1462,154329.084868
1463,167570.214780
1464,182207.404044
1465,209428.443159
...,...
2915,79574.054018
2916,88207.153250
2917,144551.503997


In [138]:
# Save submission
df_test['SalePrice'].to_csv('submission.csv')